# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-26 12:01:23 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [2]:
using DataFrames
using CSV
using Pipe
using Statistics

# Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user).


In [3]:
URL = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"
download(URL, "u.user")

"u.user"

# Step 3. Assign it to a variable called users.

In [4]:
users = CSV.read("u.user", delim = "|", DataFrame)

,user_id,age,gender,occupation,zip_code
,Int64,Int64,String1,String15,String7
1,1,24,M,technician,85711
2,2,53,F,other,94043
3,3,23,M,writer,32067
4,4,24,M,technician,43537
5,5,33,F,other,15213
6,6,42,M,executive,98101
7,7,57,M,administrator,91344
8,8,36,M,administrator,05201
9,9,29,M,student,01002


# Step 4. Discover what is the mean age per occupation

In [5]:
@pipe users |> groupby(_, :occupation) |> (_, :age => mean)

(GroupedDataFrame with 21 groups based on key: occupation
First Group (27 rows): occupation = "technician"
 Row │ user_id  age    gender   occupation  zip_code 
     │ Int64    Int64  String1  String15    String7  
─────┼───────────────────────────────────────────────
   1 │       1     24  M        technician  85711
   2 │       4     24  M        technician  43537
   3 │      44     26  M        technician  46260
   4 │      77     30  M        technician  29379
   5 │     143     42  M        technician  08832
   6 │     197     55  M        technician  75094
   7 │     244     28  M        technician  80525
   8 │     294     34  M        technician  92110
   9 │     311     32  M        technician  73071
  10 │     325     48  M        technician  02139
  11 │     441     50  M        technician  55013
  ⋮  │    ⋮       ⋮       ⋮         ⋮          ⋮
  17 │     715     21  M        technician  91206
  18 │     717     24  M        technician  84105
  19 │     718     42  M        

# Step 5. Discover the Male ratio per occupation and sort it from the most to the least

In [6]:
convert_num(x) = x == "M" ? 1 : 0
users[!,:gender_num] = convert_num.(users.gender)
users

,user_id,age,gender,occupation,zip_code,gender_num
,Int64,Int64,String1,String15,String7,Int64
1,1,24,M,technician,85711,1
2,2,53,F,other,94043,0
3,3,23,M,writer,32067,1
4,4,24,M,technician,43537,1
5,5,33,F,other,15213,0
6,6,42,M,executive,98101,1
7,7,57,M,administrator,91344,1
8,8,36,M,administrator,05201,1
9,9,29,M,student,01002,1


In [22]:
male_ratio = @pipe users |> groupby(_, :occupation) |> combine(_, :gender_num => (x -> sum(x) / length(x)) => :male_ratio)
sort(male_ratio, (:male_ratio),rev=true)

,occupation,male_ratio
,String15,Float64
1,doctor,1.0
2,engineer,0.970149
3,technician,0.962963
4,retired,0.928571
5,programmer,0.909091
6,executive,0.90625
7,scientist,0.903226
8,entertainment,0.888889
9,lawyer,0.833333


# Step 6. For each occupation, calculate the minimum and maximum ages


In [25]:
@pipe users |> groupby(_, :occupation) |> combine(_, :age => minimum, :age => maximum)

,occupation,age_minimum,age_maximum
,String15,Int64,Int64
1,technician,21,55
2,other,13,64
3,writer,18,60
4,executive,22,69
5,administrator,21,70
6,student,7,42
7,lawyer,21,53
8,educator,23,63
9,scientist,23,55


# Step 7. For each combination of occupation and gender, calculate the mean age


In [43]:
agg = combine(groupby(users, [:occupation, :gender]), :gender => length => :count)
transform(groupby(agg, [:occupation]), :count => (x -> x / sum(x) * 100) => :age_percentage)

,occupation,gender,count,age_percentage
,String15,String1,Int64,Float64
1,technician,M,26,96.2963
2,technician,F,1,3.7037
3,other,M,69,65.7143
4,other,F,36,34.2857
5,writer,M,26,57.7778
6,writer,F,19,42.2222
7,executive,M,29,90.625
8,executive,F,3,9.375
9,administrator,M,43,54.4304
